In [1]:
import pandas as pd
import numpy as np

In [4]:
games_details = pd.read_csv('data/games_details.csv')
games_details.head()

C:\Users\rhick\AppData\Local\Temp\ipykernel_24476\3199302104.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  games_details = pd.read_csv('data/games_details.csv')


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0


In [5]:
games = pd.read_csv('data/games.csv')
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


In [10]:
games.isna().sum()

GAME_DATE_EST            0
GAME_ID                  0
GAME_STATUS_TEXT         0
HOME_TEAM_ID             0
VISITOR_TEAM_ID          0
SEASON                   0
TEAM_ID_home             0
PTS_home                99
FG_PCT_home             99
FT_PCT_home             99
FG3_PCT_home            99
AST_home                99
REB_home                99
TEAM_ID_away             0
PTS_away                99
FG_PCT_away             99
FT_PCT_away             99
FG3_PCT_away            99
AST_away                99
REB_away                99
HOME_TEAM_WINS           0
HOME_TEAM_PTS_AVG    25838
AWAY_TEAM_PTS_AVG    25831
dtype: int64

In [45]:
games.sort_values('GAME_DATE_EST', inplace=True)

team_pts = {}
current_season = 2003

for index, row in games.iterrows():
    if row['SEASON'] == current_season:
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_pts = row['PTS_home']
        away_team_pts = row['PTS_away']

        if not np.isnan(home_team_pts):
            if home_team_id in team_pts:
                team_pts[home_team_id]['pts'].append(home_team_pts)
            else:
                team_pts[home_team_id] = {'pts': [home_team_pts]}

        if not np.isnan(away_team_pts):
            if away_team_id in team_pts:
                team_pts[away_team_id]['pts'].append(away_team_pts)
            else:
                team_pts[away_team_id] = {'pts': [away_team_pts]}

        if home_team_id in team_pts:
            home_team_avg_pts = np.mean(team_pts[home_team_id]['pts'])
        else:
            home_team_avg_pts = np.nan

        if away_team_id in team_pts:
            away_team_avg_pts = np.mean(team_pts[away_team_id]['pts'])
        else:
            away_team_avg_pts = np.nan

        games.loc[index, 'HOME_TEAM_PTS_AVG'] = home_team_avg_pts
        games.loc[index, 'AWAY_TEAM_PTS_AVG'] = away_team_avg_pts
    else:
        current_season = current_season + 1
        team_pts = {}
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_pts = row['PTS_home']
        away_team_pts = row['PTS_away']

        if not np.isnan(home_team_pts):
            if home_team_id in team_pts:
                team_pts[home_team_id]['pts'].append(home_team_pts)
            else:
                team_pts[home_team_id] = {'pts': [home_team_pts]}

        if not np.isnan(away_team_pts):
            if away_team_id in team_pts:
                team_pts[away_team_id]['pts'].append(away_team_pts)
            else:
                team_pts[away_team_id] = {'pts': [away_team_pts]}

        if home_team_id in team_pts:
            home_team_avg_pts = np.mean(team_pts[home_team_id]['pts'])
        else:
            home_team_avg_pts = np.nan

        if away_team_id in team_pts:
            away_team_avg_pts = np.mean(team_pts[away_team_id]['pts'])
        else:
            away_team_avg_pts = np.nan

        games.loc[index, 'HOME_TEAM_PTS_AVG'] = home_team_avg_pts
        games.loc[index, 'AWAY_TEAM_PTS_AVG'] = away_team_avg_pts

In [48]:
games.sort_values('GAME_DATE_EST', inplace=True)

team_ast = {}
current_season = 2003

for index, row in games.iterrows():
    if row['SEASON'] == current_season:
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_ast = row['AST_home']
        away_team_ast = row['AST_away']

        if not np.isnan(home_team_ast):
            if home_team_id in team_ast:
                team_ast[home_team_id]['ast'].append(home_team_ast)
            else:
                team_ast[home_team_id] = {'ast': [home_team_ast]}

        if not np.isnan(away_team_ast):
            if away_team_id in team_ast:
                team_ast[away_team_id]['ast'].append(away_team_ast)
            else:
                team_ast[away_team_id] = {'ast': [away_team_ast]}

        if home_team_id in team_ast:
            home_team_avg_ast = np.mean(team_ast[home_team_id]['ast'])
        else:
            home_team_avg_ast = np.nan

        if away_team_id in team_ast:
            away_team_avg_ast = np.mean(team_ast[away_team_id]['ast'])
        else:
            away_team_avg_ast = np.nan

        games.loc[index, 'HOME_TEAM_AST_AVG'] = home_team_avg_ast
        games.loc[index, 'AWAY_TEAM_AST_AVG'] = away_team_avg_ast
    else:
        current_season = current_season + 1
        team_ast = {}
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_ast = row['AST_home']
        away_team_ast = row['AST_away']

        if not np.isnan(home_team_ast):
            if home_team_id in team_ast:
                team_ast[home_team_id]['ast'].append(home_team_ast)
            else:
                team_ast[home_team_id] = {'ast': [home_team_ast]}

        if not np.isnan(away_team_ast):
            if away_team_id in team_ast:
                team_ast[away_team_id]['ast'].append(away_team_ast)
            else:
                team_ast[away_team_id] = {'ast': [away_team_ast]}

        if home_team_id in team_ast:
            home_team_avg_ast = np.mean(team_ast[home_team_id]['ast'])
        else:
            home_team_avg_ast = np.nan

        if away_team_id in team_ast:
            away_team_avg_ast = np.mean(team_ast[away_team_id]['ast'])
        else:
            away_team_avg_ast = np.nan

        games.loc[index, 'HOME_TEAM_AST_AVG'] = home_team_avg_ast
        games.loc[index, 'AWAY_TEAM_AST_AVG'] = away_team_avg_ast

In [53]:
games.sort_values('GAME_DATE_EST', inplace=True)

team_reb = {}
current_season = 2003

for index, row in games.iterrows():
    if row['SEASON'] == current_season:
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_reb = row['REB_home']
        away_team_reb = row['REB_away']

        if not np.isnan(home_team_reb):
            if home_team_id in team_reb:
                team_reb[home_team_id]['reb'].append(home_team_reb)
            else:
                team_reb[home_team_id] = {'reb': [home_team_reb]}

        if not np.isnan(away_team_reb):
            if away_team_id in team_reb:
                team_reb[away_team_id]['reb'].append(away_team_reb)
            else:
                team_reb[away_team_id] = {'reb': [away_team_reb]}

        if home_team_id in team_reb:
            home_team_avg_reb = np.mean(team_reb[home_team_id]['reb'])
        else:
            home_team_avg_reb = np.nan

        if away_team_id in team_reb:
            away_team_avg_reb = np.mean(team_reb[away_team_id]['reb'])
        else:
            away_team_avg_reb = np.nan

        games.loc[index, 'HOME_TEAM_REB_AVG'] = home_team_avg_reb
        games.loc[index, 'AWAY_TEAM_REB_AVG'] = away_team_avg_reb
    else:
        current_season = current_season + 1
        team_reb = {}
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_reb = row['REB_home']
        away_team_reb = row['REB_away']

        if not np.isnan(home_team_reb):
            if home_team_id in team_reb:
                team_reb[home_team_id]['reb'].append(home_team_reb)
            else:
                team_reb[home_team_id] = {'reb': [home_team_reb]}

        if not np.isnan(away_team_reb):
            if away_team_id in team_reb:
                team_reb[away_team_id]['reb'].append(away_team_reb)
            else:
                team_reb[away_team_id] = {'reb': [away_team_reb]}

        if home_team_id in team_reb:
            home_team_avg_reb = np.mean(team_reb[home_team_id]['reb'])
        else:
            home_team_avg_reb = np.nan

        if away_team_id in team_reb:
            away_team_avg_reb = np.mean(team_reb[away_team_id]['reb'])
        else:
            away_team_avg_reb = np.nan

        games.loc[index, 'HOME_TEAM_REB_AVG'] = home_team_avg_reb
        games.loc[index, 'AWAY_TEAM_REB_AVG'] = away_team_avg_reb

In [60]:
games.sort_values('GAME_DATE_EST', inplace=True)

team_wins = {}
current_season = 2003

for index, row in games.iterrows():
    if row['SEASON'] == current_season:
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_wins = row['HOME_TEAM_WINS']
        away_team_wins = abs(row['HOME_TEAM_WINS']-1)

        if not np.isnan(home_team_wins):
            if home_team_id in team_wins:
                team_wins[home_team_id]['wins'].append(home_team_wins)
            else:
                team_wins[home_team_id] = {'wins': [home_team_wins]}

        if not np.isnan(away_team_wins):
            if away_team_id in team_wins:
                team_wins[away_team_id]['wins'].append(away_team_wins)
            else:
                team_wins[away_team_id] = {'wins': [away_team_wins]}

        if home_team_id in team_wins:
            home_team_avg_wins = np.mean(team_wins[home_team_id]['wins'])
        else:
            home_team_avg_wins = np.nan

        if away_team_id in team_wins:
            away_team_avg_wins = np.mean(team_wins[away_team_id]['wins'])
        else:
            away_team_avg_wins = np.nan

        games.loc[index, 'HOME_TEAM_WIN_PCT'] = home_team_avg_wins
        games.loc[index, 'AWAY_TEAM_WIN_PCT'] = away_team_avg_wins
    else:
        current_season = current_season + 1
        team_wins = {}
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_wins = row['HOME_TEAM_WINS']
        away_team_wins = abs(row['HOME_TEAM_WINS']-1)

        if not np.isnan(home_team_wins):
            if home_team_id in team_wins:
                team_wins[home_team_id]['wins'].append(home_team_wins)
            else:
                team_wins[home_team_id] = {'wins': [home_team_wins]}

        if not np.isnan(away_team_wins):
            if away_team_id in team_wins:
                team_wins[away_team_id]['wins'].append(away_team_wins)
            else:
                team_wins[away_team_id] = {'wins': [away_team_wins]}

        if home_team_id in team_wins:
            home_team_avg_wins = np.mean(team_wins[home_team_id]['wins'])
        else:
            home_team_avg_wins = np.nan

        if away_team_id in team_wins:
            away_team_avg_wins = np.mean(team_wins[away_team_id]['wins'])
        else:
            away_team_avg_wins = np.nan

        games.loc[index, 'HOME_TEAM_WIN_PCT'] = home_team_avg_wins
        games.loc[index, 'AWAY_TEAM_WIN_PCT'] = away_team_avg_wins

In [64]:
games.sort_values('GAME_DATE_EST', inplace=True)

team_fg = {}
current_season = 2003

for index, row in games.iterrows():
    if row['SEASON'] == current_season:
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_fg = row['FG_PCT_home']
        away_team_fg = row['FG_PCT_away']

        if not np.isnan(home_team_fg):
            if home_team_id in team_fg:
                team_fg[home_team_id]['fg'].append(home_team_fg)
            else:
                team_fg[home_team_id] = {'fg': [home_team_fg]}

        if not np.isnan(away_team_fg):
            if away_team_id in team_fg:
                team_fg[away_team_id]['fg'].append(away_team_fg)
            else:
                team_fg[away_team_id] = {'fg': [away_team_fg]}

        if home_team_id in team_fg:
            home_team_avg_fg = np.mean(team_fg[home_team_id]['fg'])
        else:
            home_team_avg_fg = np.nan

        if away_team_id in team_fg:
            away_team_avg_fg = np.mean(team_fg[away_team_id]['fg'])
        else:
            away_team_avg_fg = np.nan

        games.loc[index, 'HOME_TEAM_FG_PCT_AVG'] = home_team_avg_fg
        games.loc[index, 'AWAY_TEAM_FG_PCT_AVG'] = away_team_avg_fg
    else:
        current_season = current_season + 1
        team_fg = {}
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_fg = row['FG_PCT_home']
        away_team_fg = row['FG_PCT_away']

        if not np.isnan(home_team_fg):
            if home_team_id in team_fg:
                team_fg[home_team_id]['fg'].append(home_team_fg)
            else:
                team_fg[home_team_id] = {'fg': [home_team_fg]}

        if not np.isnan(away_team_fg):
            if away_team_id in team_fg:
                team_fg[away_team_id]['fg'].append(away_team_fg)
            else:
                team_fg[away_team_id] = {'fg': [away_team_fg]}

        if home_team_id in team_fg:
            home_team_avg_fg = np.mean(team_fg[home_team_id]['fg'])
        else:
            home_team_avg_fg = np.nan

        if away_team_id in team_fg:
            away_team_avg_fg = np.mean(team_fg[away_team_id]['fg'])
        else:
            away_team_avg_fg = np.nan

        games.loc[index, 'HOME_TEAM_FG_PCT_AVG'] = home_team_avg_fg
        games.loc[index, 'AWAY_TEAM_FG_PCT_AVG'] = away_team_avg_fg

In [68]:
games.sort_values('GAME_DATE_EST', inplace=True)

team_ft = {}
current_season = 2003

for index, row in games.iterrows():
    if row['SEASON'] == current_season:
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_ft = row['FT_PCT_home']
        away_team_ft = row['FT_PCT_away']

        if not np.isnan(home_team_ft):
            if home_team_id in team_ft:
                team_ft[home_team_id]['ft'].append(home_team_ft)
            else:
                team_ft[home_team_id] = {'ft': [home_team_ft]}

        if not np.isnan(away_team_ft):
            if away_team_id in team_ft:
                team_ft[away_team_id]['ft'].append(away_team_ft)
            else:
                team_ft[away_team_id] = {'ft': [away_team_ft]}

        if home_team_id in team_ft:
            home_team_avg_ft = np.mean(team_ft[home_team_id]['ft'])
        else:
            home_team_avg_ft = np.nan

        if away_team_id in team_ft:
            away_team_avg_ft = np.mean(team_ft[away_team_id]['ft'])
        else:
            away_team_avg_ft = np.nan

        games.loc[index, 'HOME_TEAM_FT_PCT_AVG'] = home_team_avg_ft
        games.loc[index, 'AWAY_TEAM_FT_PCT_AVG'] = away_team_avg_ft
    else:
        current_season = current_season + 1
        team_ft = {}
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_ft = row['FT_PCT_home']
        away_team_ft = row['FT_PCT_away']

        if not np.isnan(home_team_ft):
            if home_team_id in team_ft:
                team_ft[home_team_id]['ft'].append(home_team_ft)
            else:
                team_ft[home_team_id] = {'ft': [home_team_ft]}

        if not np.isnan(away_team_ft):
            if away_team_id in team_ft:
                team_ft[away_team_id]['ft'].append(away_team_ft)
            else:
                team_ft[away_team_id] = {'ft': [away_team_ft]}

        if home_team_id in team_ft:
            home_team_avg_ft = np.mean(team_ft[home_team_id]['ft'])
        else:
            home_team_avg_ft = np.nan

        if away_team_id in team_ft:
            away_team_avg_ft = np.mean(team_ft[away_team_id]['ft'])
        else:
            away_team_avg_ft = np.nan

        games.loc[index, 'HOME_TEAM_FT_PCT_AVG'] = home_team_avg_ft
        games.loc[index, 'AWAY_TEAM_FT_PCT_AVG'] = away_team_avg_ft

In [70]:
games.sort_values('GAME_DATE_EST', inplace=True)

team_fg3 = {}
current_season = 2003

for index, row in games.iterrows():
    if row['SEASON'] == current_season:
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_fg3 = row['FG3_PCT_home']
        away_team_fg3 = row['FG3_PCT_away']

        if not np.isnan(home_team_fg3):
            if home_team_id in team_fg3:
                team_fg3[home_team_id]['fg3'].append(home_team_fg3)
            else:
                team_fg3[home_team_id] = {'fg3': [home_team_fg3]}

        if not np.isnan(away_team_fg3):
            if away_team_id in team_fg3:
                team_fg3[away_team_id]['fg3'].append(away_team_fg3)
            else:
                team_fg3[away_team_id] = {'fg3': [away_team_fg3]}

        if home_team_id in team_fg3:
            home_team_avg_fg3 = np.mean(team_fg3[home_team_id]['fg3'])
        else:
            home_team_avg_fg3 = np.nan

        if away_team_id in team_fg3:
            away_team_avg_fg3 = np.mean(team_fg3[away_team_id]['fg3'])
        else:
            away_team_avg_fg3 = np.nan

        games.loc[index, 'HOME_TEAM_FG3_PCT_AVG'] = home_team_avg_fg3
        games.loc[index, 'AWAY_TEAM_FG3_PCT_AVG'] = away_team_avg_fg3
    else:
        current_season = current_season + 1
        team_fg3 = {}
        home_team_id = row['HOME_TEAM_ID']
        away_team_id = row['VISITOR_TEAM_ID']
        home_team_fg3 = row['FG3_PCT_home']
        away_team_fg3 = row['FG3_PCT_away']

        if not np.isnan(home_team_fg3):
            if home_team_id in team_fg3:
                team_fg3[home_team_id]['fg3'].append(home_team_fg3)
            else:
                team_fg3[home_team_id] = {'fg3': [home_team_fg3]}

        if not np.isnan(away_team_fg3):
            if away_team_id in team_fg3:
                team_fg3[away_team_id]['fg3'].append(away_team_fg3)
            else:
                team_fg3[away_team_id] = {'fg3': [away_team_fg3]}

        if home_team_id in team_fg3:
            home_team_avg_fg3 = np.mean(team_fg3[home_team_id]['fg3'])
        else:
            home_team_avg_fg3 = np.nan

        if away_team_id in team_fg3:
            away_team_avg_fg3 = np.mean(team_fg3[away_team_id]['fg3'])
        else:
            away_team_avg_fg3 = np.nan

        games.loc[index, 'HOME_TEAM_FG3_PCT_AVG'] = home_team_avg_fg3
        games.loc[index, 'AWAY_TEAM_FG3_PCT_AVG'] = away_team_avg_fg3

In [76]:
display(games.tail(545))

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,HOME_TEAM_REB_AVG,AWAY_TEAM_REB_AVG,HOME_TEAM_WIN_PCT,AWAY_TEAM_WIN_PCT,HOME_TEAM_FG_PCT_AVG,AWAY_TEAM_FG_PCT_AVG,HOME_TEAM_FT_PCT_AVG,AWAY_TEAM_FT_PCT_AVG,HOME_TEAM_FG3_PCT_AVG,AWAY_TEAM_FG3_PCT_AVG
544,2022-06-10,42100404,Final,1610612738,1610612744,2021,1610612738,97.0,0.400,0.737,...,45.379630,45.345794,0.620370,0.672897,0.462407,0.471318,0.814083,0.768561,0.356194,0.367075
543,2022-06-13,42100405,Final,1610612744,1610612738,2021,1610612744,104.0,0.466,0.867,...,45.287037,45.394495,0.675926,0.614679,0.471269,0.461954,0.769472,0.812826,0.365759,0.356083
542,2022-06-16,42100406,Final,1610612738,1610612744,2021,1610612738,90.0,0.425,0.917,...,45.354545,45.275229,0.609091,0.678899,0.461618,0.470734,0.813773,0.771587,0.356418,0.366193
541,2022-09-30,12200001,Final,1610612764,1610612744,2022,1610612764,87.0,0.369,0.633,...,44.000000,62.000000,0.000000,1.000000,0.369000,0.367000,0.633000,0.775000,0.171000,0.269000
540,2022-10-01,12200003,Final,1610612749,1610612763,2022,1610612749,102.0,0.430,0.714,...,50.000000,41.000000,0.000000,1.000000,0.430000,0.494000,0.714000,0.724000,0.351000,0.357000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2022-12-21,22200469,Final,1610612738,1610612754,2022,1610612738,112.0,0.386,0.840,...,44.333333,42.628571,0.666667,0.485714,0.478083,0.458914,0.834056,0.787629,0.378750,0.358114
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,44.055556,42.800000,0.500000,0.457143,0.466778,0.489343,0.821028,0.822486,0.339972,0.363000
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,41.500000,43.289474,0.647059,0.210526,0.468912,0.443474,0.823324,0.781289,0.374000,0.338132
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,44.000000,43.270270,0.512821,0.351351,0.469000,0.467919,0.753538,0.759189,0.366026,0.335784


In [77]:
games.columns

Index(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS', 'HOME_TEAM_PTS_AVG', 'AWAY_TEAM_PTS_AVG',
       'HOME_TEAM_AST_AVG', 'AWAY_TEAM_AST_AVG', 'HOME_TEAM_REB_AVG',
       'AWAY_TEAM_REB_AVG', 'HOME_TEAM_WIN_PCT', 'AWAY_TEAM_WIN_PCT',
       'HOME_TEAM_FG_PCT_AVG', 'AWAY_TEAM_FG_PCT_AVG', 'HOME_TEAM_FT_PCT_AVG',
       'AWAY_TEAM_FT_PCT_AVG', 'HOME_TEAM_FG3_PCT_AVG',
       'AWAY_TEAM_FG3_PCT_AVG'],
      dtype='object')